In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from integral_timber_joints.planning import load_pddlstream

Using pddlstream from C:\Users\harry\Documents\code_ws\pb_ws\coop_assembly\external\pddlstream\pddlstream\__init__.py
Using strips (pyplanners) from C:\Users\harry\Documents\code_ws\pb_ws\coop_assembly\external\pyplanners\strips\__init__.py


## Parse ITJ process

In [3]:
import os
import time
from termcolor import cprint
import pybullet_planning as pp
from pybullet_planning import elapsed_time
from integral_timber_joints.planning.parsing import parse_process, save_process_and_movements, \
    get_process_path, save_process

In [4]:
# parse options
design_dir = '210916_SymbolicPlanning' # '210419_AnticlasticShelter' # '210605_ScrewdriverTestProcess' # 210419_AnticlasticShelter
problem = 'nine_pieces_process.json' #'shelter_process.json' # 'shelter_process.json' pavilion_process.json' # 'twelve_pieces_process.json'
problem_subdir = '.'

In [5]:
process = parse_process(design_dir, problem, subdir=problem_subdir)

# Double check entire solution is valid
for beam_id in process.assembly.sequence:
    if not process.dependency.beam_all_valid(beam_id):
        print('Yay')
        process.dependency.compute_all(beam_id)
        assert process.dependency.beam_all_valid(beam_id)

Process json parsed from c:\users\harry\dropbox (mit)\code_ws_dropbox\itj_ws\integral_timber_joints\external\itj_design_study\210916_SymbolicPlanning\nine_pieces_process.json


## Solve

In [6]:
discrete_planner = {
    'search': 'eager', # eager | lazy
    # lazy might be faster but the solution quality might be low
    'evaluator': 'greedy',
    'heuristic': 'goal', # goal | add | ff
    #'heuristic': ['ff', get_bias_fn(element_from_index)],
    'successors': 'all', # all | random
    #'successors': get_order_fn(element_from_index),
    # ? ordering scaffolds can come here
    # TODO: confirm that this is working correctly
}

In [83]:
from integral_timber_joints.planning.robot_setup import load_RFL_world
from integral_timber_joints.planning.run import set_initial_state

# * Connect to path planning backend and initialize robot parameters
# viewer or diagnosis or view_states or watch or step_sim,
client, robot, _ = load_RFL_world(viewer=1, verbose=False)
set_initial_state(client, robot, process, disable_env=False, reinit_tool=False)

In [82]:
client.disconnect()

In [92]:
from pddlstream.algorithms.downward import set_cost_scale, parse_action
from pddlstream.algorithms.meta import solve
from pddlstream.utils import INF
from pddlstream.language.constants import print_plan, is_plan

from integral_timber_joints.planning.state import set_state
from integral_timber_joints.planning.pddlstream_definitions.problem import get_pddlstream_problem
from integral_timber_joints.planning.pddlstream_definitions.postprocessing import print_itj_pddl_plan

debug = 0
options = {
    'avoid_collisions' : False,
    'verbose' : True,
    'debug' : debug,
    'diagnosis' : debug,
    'distance_threshold' : 0.02,
}

set_state(client, robot, process, process.initial_state, initialize=False)
pddlstream_problem = get_pddlstream_problem(client, process, robot, use_partial_order=1, 
                                            debug=0, reset_to_home=1, 
                                            consider_transition=0, options=options)

# print('Init:', pddlstream_problem.init)
print()
print('Goal:', pddlstream_problem.goal)
print()

costs = True
set_cost_scale(1)

start_time = time.time()
solution = solve(pddlstream_problem, algorithm='incremental', #adaptive
                 max_time=60,
                 unit_costs=True,
                 max_planner_time=300, 
                 debug=0, verbose=0)
# , planner=discrete_planner)
cprint('Total solve time: {:.3f}'.format(elapsed_time(start_time)), 'blue')

plan, cost, evaluations = solution
plan_success = is_plan(plan)
cprint('Planning {}'.format('succeeds' if plan_success else 'fails'), 'green' if plan_success else 'red')

print('-'*10)
print_itj_pddl_plan(plan, 0)


Goal: ('and', ('Assembled', 'b0'), ('Assembled', 'b1'), ('Assembled', 'b2'), ('Assembled', 'b3'), ('Assembled', 'b5'), ('Assembled', 'b6'), ('Assembled', 'b7'), ('Assembled', 'b8'), ('Assembled', 'b4'), ('AtRack', 'c1'), ('AtRack', 'c2'), ('AtRack', 'c3'), ('AtRack', 'c4'), ('AtRack', 'g1'), ('AtRack', 'g2'), ('AtRack', 'g3'))

Iteration: 1 | Complexity: 0 | Calls: 0 | Evaluations: 278 | Solved: False | Cost: inf | Search Time: 0.010 | Sample Time: 0.000 | Time: 0.010
Initial: State{Atom atpose(v37, v38), Atom atpose(v40, v41), Atom atpose(v43, v44), Atom atpose(v46, v47), Atom atpose(v59, v60), Atom atpose(v62, v63), Atom atpose(v65, v66), Atom notoolatjoint(v1, v13), Atom notoolatjoint(v1, v21), Atom notoolatjoint(v1, v9), Atom notoolatjoint(v13, v1), Atom notoolatjoint(v13, v5), Atom notoolatjoint(v17, v21), Atom notoolatjoint(v17, v33), Atom notoolatjoint(v21, v1), Atom notoolatjoint(v21, v17), Atom notoolatjoint(v25, v29), Atom notoolatjoint(v25, v33), Atom notoolatjoint(v29, v25

In [50]:
from pddlstream.algorithms.downward import is_valid_plan, apply_action, task_from_domain_problem, get_problem, \
    get_action_instances
from pddlstream.algorithms.algorithm import parse_problem
from pddlstream.algorithms.constraints import PlanConstraints

# is_valid_plan(pddlstream_problem.init, plan)
# task = task_from_domain_problem(pddlstream_problem, get_problem(evaluations, goal_expression, domain, unit_costs=True))

unit_costs = True
evaluations, goal_expression, domain, externals = parse_problem(
    pddlstream_problem, constraints=PlanConstraints(), unit_costs=unit_costs)

In [59]:
from pddlstream.algorithms.refinement import optimistic_process_streams
from pddlstream.algorithms.scheduling.recover_streams import evaluations_from_stream_plan
from pddlstream.algorithms.instantiate_task import instantiate_task

results, exhausted = optimistic_process_streams(evaluations, externals, complexity_limit=INF, max_effort=None)
exh_evaluations = evaluations_from_stream_plan(evaluations, results, max_effort=None)

task = task_from_domain_problem(domain, get_problem(exh_evaluations, goal_expression, domain, unit_costs))

In [60]:
instantiated = instantiate_task(task)


Generating Datalog program... [0.000s CPU, 0.002s wall-clock]
Normalizing Datalog program...
Normalizing Datalog program: [0.000s CPU, 0.002s wall-clock]
Preparing model... [0.000s CPU, 0.003s wall-clock]
Generated 74 rules.
Computing model... [0.031s CPU, 0.030s wall-clock]
2180 relevant atoms
171 auxiliary atoms
2351 final queue length
7301 total queue pushes
Completing instantiation... [0.047s CPU, 0.040s wall-clock]
Infeasible: False
Instantiation time: 0.082s
Instantiated frequencies:
Atoms: {alltoolatjoints: 30, assembled: 9, atrack: 16, attached: 16, canfreemove: 1, connected: 30, new-axiom@0: 8, new-axiom@1: 20, new-axiom@2: 9, notoolatjoint: 20, robotatconf: 9, robotgripperempty: 1, robottoolchangerempty: 1, toolatjoint: 140, toolnotoccupiedonjoint: 7}
Actions: {move: 81, pick_clamp_from_joint: 80, pick_clamp_from_rack: 4, pick_element_from_rack: 27, pick_gripper_from_rack: 3, place_clamp_at_joint: 160, place_element_on_structure: 39, place_tool_at_rack: 11}
Axioms: {alltoola

In [68]:
certificates.all_facts[0]

('robotconf', Conf-Home)

In [65]:
type(instantiated.task.init[0])

pddl.conditions.Atom

In [74]:
plan[0]

Action(name='pick_gripper_from_rack', args=('g1', @conf126, @conf226, @traj27))

In [81]:
from pddlstream.language.conversion import is_atom, is_negated_atom, objects_from_evaluations, pddl_from_object, \
    pddl_list_from_expression, obj_from_pddl

# pddl_from_object()
type(obj_from_pddl('#o1'))

pddlstream.language.object.OptimisticObject

In [82]:
instantiated.actions

[<PropositionalAction '(move #o1 #o1 #o0)' at 0x222822cc1c8>,
 <PropositionalAction '(move #o1 #o10 #o0)' at 0x222822cce88>,
 <PropositionalAction '(move #o1 #o11 #o0)' at 0x222822ccb88>,
 <PropositionalAction '(move #o1 #o2 #o0)' at 0x222822ccb48>,
 <PropositionalAction '(move #o1 #o4 #o0)' at 0x222822cc988>,
 <PropositionalAction '(move #o1 #o5 #o0)' at 0x222822cc648>,
 <PropositionalAction '(move #o1 #o7 #o0)' at 0x222822cc3c8>,
 <PropositionalAction '(move #o1 #o8 #o0)' at 0x222822cc248>,
 <PropositionalAction '(move #o1 v1 #o0)' at 0x222822cc948>,
 <PropositionalAction '(move #o10 #o1 #o0)' at 0x222822cc488>,
 <PropositionalAction '(move #o10 #o10 #o0)' at 0x2228217de08>,
 <PropositionalAction '(move #o10 #o11 #o0)' at 0x2228217db88>,
 <PropositionalAction '(move #o10 #o2 #o0)' at 0x2228217d8c8>,
 <PropositionalAction '(move #o10 #o4 #o0)' at 0x2228217d9c8>,
 <PropositionalAction '(move #o10 #o5 #o0)' at 0x2228229eb08>,
 <PropositionalAction '(move #o10 #o7 #o0)' at 0x222822d5b08>

In [85]:
from pddlstream.algorithms.downward import parse_action

# parse_action(instantiated.actions[0].name)
obj_from_pddl(parse_action(instantiated.actions[-1].name).args[0])

c4

In [69]:
instantiated.actions[0]

<PropositionalAction '(move #o1 #o1 #o0)' at 0x222822cc1c8>

In [63]:
instantiated.task.init

[<Atom robotconf(v1)>,
 <Atom robotatconf(v1)>,
 <Atom canfreemove()>,
 <Atom robottoolchangerempty()>,
 <Atom element(v2)>,
 <Atom atrack(v2)>,
 <Atom iselement(v2)>,
 <Atom grounded(v2)>,
 <Atom element(v3)>,
 <Atom atrack(v3)>,
 <Atom iselement(v3)>,
 <Atom grounded(v3)>,
 <Atom element(v4)>,
 <Atom atrack(v4)>,
 <Atom iselement(v4)>,
 <Atom element(v5)>,
 <Atom atrack(v5)>,
 <Atom iselement(v5)>,
 <Atom element(v6)>,
 <Atom atrack(v6)>,
 <Atom iselement(v6)>,
 <Atom grounded(v6)>,
 <Atom element(v7)>,
 <Atom atrack(v7)>,
 <Atom iselement(v7)>,
 <Atom element(v8)>,
 <Atom atrack(v8)>,
 <Atom iselement(v8)>,
 <Atom grounded(v8)>,
 <Atom element(v9)>,
 <Atom atrack(v9)>,
 <Atom iselement(v9)>,
 <Atom element(v10)>,
 <Atom atrack(v10)>,
 <Atom iselement(v10)>,
 <Atom joint(v2, v4)>,
 <Atom joint(v4, v2)>,
 <Atom notoolatjoint(v2, v4)>,
 <Atom notoolatjoint(v4, v2)>,
 <Atom joint(v2, v5)>,
 <Atom joint(v5, v2)>,
 <Atom notoolatjoint(v2, v5)>,
 <Atom notoolatjoint(v5, v2)>,
 <Atom joint(

In [91]:
from pddlstream.language.conversion import value_from_evaluation

solution.certificate.all_facts

[('robotconf', Conf-Home),
 ('robotatconf', Conf-Home),
 ('canfreemove',),
 ('robottoolchangerempty',),
 ('element', 'b0'),
 ('atrack', 'b0'),
 ('iselement', 'b0'),
 ('grounded', 'b0'),
 ('element', 'b1'),
 ('atrack', 'b1'),
 ('iselement', 'b1'),
 ('grounded', 'b1'),
 ('element', 'b2'),
 ('atrack', 'b2'),
 ('iselement', 'b2'),
 ('element', 'b3'),
 ('atrack', 'b3'),
 ('iselement', 'b3'),
 ('element', 'b5'),
 ('atrack', 'b5'),
 ('iselement', 'b5'),
 ('grounded', 'b5'),
 ('element', 'b6'),
 ('atrack', 'b6'),
 ('iselement', 'b6'),
 ('element', 'b7'),
 ('atrack', 'b7'),
 ('iselement', 'b7'),
 ('grounded', 'b7'),
 ('element', 'b8'),
 ('atrack', 'b8'),
 ('iselement', 'b8'),
 ('element', 'b4'),
 ('atrack', 'b4'),
 ('iselement', 'b4'),
 ('joint', 'b0', 'b2'),
 ('joint', 'b2', 'b0'),
 ('notoolatjoint', 'b0', 'b2'),
 ('notoolatjoint', 'b2', 'b0'),
 ('joint', 'b0', 'b3'),
 ('joint', 'b3', 'b0'),
 ('notoolatjoint', 'b0', 'b3'),
 ('notoolatjoint', 'b3', 'b0'),
 ('joint', 'b0', 'b6'),
 ('joint', 'b6'

# Sequence

In [23]:
place_actions = [action for action in plan if action.name == 'place_element_on_structure']
element_sequence = [ac.args[0] for ac in place_actions]

In [24]:
process.assembly.sequence

['b0', 'b1', 'b2', 'b3', 'b5', 'b6', 'b7', 'b8', 'b4']

In [25]:
element_sequence

['b0', 'b1', 'b2', 'b3', 'b5', 'b6', 'b7', 'b8', 'b4']

In [26]:
place_actions

[Action(name='place_element_on_structure', args=('b0', Frame(Point(19016.343, 9160.006, 279.218), Vector(0.000, 0.000, 1.000), Vector(-1.000, 0.000, 0.000)), Frame(Point(25496.239, 7136.104, 579.218), Vector(0.000, 1.000, 0.000), Vector(-1.000, 0.000, 0.000)), 'g1', Frame(Point(23439.739, 4480.771, 990.818), Vector(-0.000, 1.000, 0.000), Vector(1.000, 0.000, -0.000)), @conf335)),
 Action(name='place_element_on_structure', args=('b1', Frame(Point(19016.343, 8260.006, 3361.742), Vector(0.000, 0.000, -1.000), Vector(-1.000, 0.000, -0.000)), Frame(Point(25496.239, 7068.375, 579.218), Vector(0.000, 1.000, 0.000), Vector(-1.000, 0.000, 0.000)), 'g1', Frame(Point(23439.739, 4480.771, 990.818), Vector(-0.000, 1.000, 0.000), Vector(1.000, 0.000, -0.000)), @conf338)),
 Action(name='place_element_on_structure', args=('b2', Frame(Point(19016.343, 9455.953, 1810.165), Vector(-0.000, -0.986, -0.165), Vector(-1.000, 0.000, 0.000)), Frame(Point(25496.239, 7251.367, 579.218), Vector(0.000, 1.000, 0.000

# Visualize sequence in a pybullet window

In [101]:
from integral_timber_joints.planning.pddlstream_definitions.postprocessing import actions_from_pddlstream_plan

actions_from_pddlstream_plan(process, plan, verbose=1)

operator_load_element_on_rack b0 (frm)
pick_tool_from_rack g2 (frm) (tf) (conf)
pick_element_from_rack b0 (frm) (tf) g2 (tf) (conf)
operator_load_element_on_rack b1 (frm)
place_element_on_structure b0 (frm) (tf) g2 (tf) (conf)
Beam id: b0
|- Operator load Beam ('b0') for pickup
|- Pick PG1000 ('g2') from Storage
Configuration((20.832, -2.102, -3.796, -0.816, -0.623, -0.418, -0.001, -0.531, -0.755), (2, 2, 2, 0, 0, 0, 0, 0, 0), ('bridge1_joint_EA_X', 'robot11_joint_EA_Y', 'robot11_joint_EA_Z', 'robot11_joint_1', 'robot11_joint_2', 'robot11_joint_3', 'robot11_joint_4', 'robot11_joint_5', 'robot11_joint_6'))
|- Pick Beam ('b0') from PickupPoint using Gripper(g2)
Configuration((21.348, -6.794, -2.383, 0.232, -0.351, 0.334, -1.504, 0.232, -1.640), (2, 2, 2, 0, 0, 0, 0, 0, 0), ('bridge1_joint_EA_X', 'robot11_joint_EA_Y', 'robot11_joint_EA_Z', 'robot11_joint_1', 'robot11_joint_2', 'robot11_joint_3', 'robot11_joint_4', 'robot11_joint_5', 'robot11_joint_6'))
|- Operator load Beam ('b1') for pic

1

In [105]:
from integral_timber_joints.planning.robot_setup import load_RFL_world
from integral_timber_joints.planning.run import set_initial_state

# * Connect to path planning backend and initialize robot parameters
# viewer or diagnosis or view_states or watch or step_sim,
client, robot, _ = load_RFL_world(viewer=1, verbose=False)
set_initial_state(client, robot, process, disable_env=False, reinit_tool=False)

In [107]:
client.disconnect()

In [106]:
from integral_timber_joints.planning.state import set_state
from integral_timber_joints.process import RoboticMovement, RobotClampAssemblyProcess

for m in process.movements:
    # print(m.short_summary)
    start_state = process.get_movement_start_scene(m)
    end_state = process.get_movement_end_scene(m)  
    set_state(client, robot, process, end_state, initialize=False)
#     pp.wait_if_gui('End state.')
    pp.wait_for_duration(0.1)
print('Finished.')

OperatorLoadBeamMovement(#A0_M0, Opeartor Load Beam (b0) to Pickup Location)
RoboticFreeMovement(#A0_M0, Free Move reach Storage Approach Frame of PG1000 ('g2'), to get tool., traj 0)
RoboticLinearMovement(#A0_M1, Linear Advance to Storage Frame of PG1000 ('g2'), to get tool., traj 0)
RoboticDigitalOutput(#A0_M2, Toolchanger Lock PG1000 ('g2'))
RoboticDigitalOutput(#A0_M3, PG1000 ('g2') Open Gripper to release itself from storage pad.)
RoboticLinearMovement(#A0_M4, Linear Retract after getting PG1000 ('g2') from storage., traj 0)
RoboticFreeMovement(#A0_M0, Free Move to reach Pickup Approach Frame of Beam ('b0'), traj 0)
RoboticDigitalOutput(#A0_M1, Gripper ('g2') Open Gripper before gripping Beam ('b0'))
RoboticLinearMovement(#A0_M2, Linear Advance to Storage Frame of Beam ('b0'), traj 0)
RoboticDigitalOutput(#A0_M3, Gripper ('g2') Close Gripper to grip Beam ('b0'))
RoboticLinearMovement(#A0_M4, Linear Retract after picking up Beam ('b0'), traj 0)
OperatorLoadBeamMovement(#A0_M0, Opea

## Construct symbolic PDDL problem

In [46]:
from integral_timber_joints.planning.pddlstream_definitions.problem import get_pddlstream_problem
from integral_timber_joints.planning.pddlstream_definitions import ITJ_PDDLSTREAM_DEF_DIR

In [65]:
import json
process_sym_data = process.to_symbolic_problem_data()
debug_problem_name = problem.split(".")[0] + "_symbolic.json"
with open(os.path.join(ITJ_PDDLSTREAM_DEF_DIR, "caelan", debug_problem_name), 'w') as f:
    json.dump(process_sym_data, f, indent=True, sort_keys=True)

In [69]:
from integral_timber_joints.planning.pddlstream_definitions.caelan.run import get_itj_pddl_problem_from_json

debug_pddl_problem = get_itj_pddl_problem_from_json(debug_problem_name, use_partial_order=True, debug=True)

print()
print('Goal:', debug_pddl_problem.goal)
print()

costs = True
set_cost_scale(1)

solution = solve(debug_pddl_problem, algorithm='adaptive', #incremental
                 max_time=60,
                 unit_costs=True,
                 max_planner_time=300, 
                 debug=0, verbose=0) #, planner=discrete_planner)

plan, cost, evaluations = solution
plan_success = is_plan(plan)
cprint('Planning {}'.format('succeeds' if plan_success else 'fails'), 'green' if plan_success else 'red')

print('-'*10)
print_plan(plan)

Symbolic process json parsed from c:\users\harry\dropbox (mit)\code_ws_dropbox\itj_ws\integral_timber_joints\src\integral_timber_joints\planning\pddlstream_definitions\caelan\nine_pieces_process_symbolic.json

Goal: ('and', ('Assembled', 'b0'), ('Assembled', 'b1'), ('Assembled', 'b2'), ('Assembled', 'b3'), ('Assembled', 'b5'), ('Assembled', 'b6'), ('Assembled', 'b7'), ('Assembled', 'b8'), ('Assembled', 'b4'))


Iteration: 1 | Complexity: 0 | Skeletons: 0 | Skeleton Queue: 0 | Disabled: 0 | Evaluations: 161 | Eager Calls: 0 | Cost: inf | Search Time: 0.001 | Sample Time: 0.000 | Total Time: 0.001
Attempt: 1 | Results: 0 | Depth: 0 | Success: False | Time: 0.120
Stream plan (inf, 0, inf): None
Action plan (inf, inf): None
No plan: increasing complexity from 0 to 1

Iteration: 2 | Complexity: 1 | Skeletons: 0 | Skeleton Queue: 0 | Disabled: 0 | Evaluations: 161 | Eager Calls: 0 | Cost: inf | Search Time: 0.121 | Sample Time: 0.000 | Total Time: 0.121


RuntimeError: Preimage fact ('assembled', b3) is not achievable!

# Misc

In [25]:
# from pddlstream.algorithms.algorithm import parse_problem
# from pddlstream.algorithms.constraints import PlanConstraints
# from pddlstream.algorithms.downward import get_problem, task_from_domain_problem
# from pddlstream.algorithms.meta import examine_instantiated

# pddlstream_problem = get_pddlstream_problem(process, use_partial_order=True, debug=False,
#                                             reset_to_home=False)

# print(pddlstream_problem.goal)

# results, instantiated = examine_instantiated(pddlstream_problem, unit_costs=1, verbose=0, debug=1)